In [12]:
# Script to write out isentropic-binned data from WRF output.
# 
# Assumes output is in a single netcdf file on pressure levels.
# 
# James Ruppert  
# jruppert@ou.edu  
# 1/28/25

import numpy as np
import xarray as xr
from thermo_functions import *
from post_proc_functions import *
from mpi4py import MPI
from write_isentrop_functions import *

# MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()